In [40]:
import numpy as np
import pandas as pd
from functools import reduce
from copy import deepcopy
import qgrid

In [2]:
def skip_i(x,i):
    if len(x.shape)>1:
        return np.concatenate([x[:,:i],x[:,i+1:]],axis=1)
    if len(x.shape) == 1:
        return np.concatenate([x[:i],x[i+1:]],axis=0)
    else:
        raise NameError('Invalid input dimetion')
        
def rec_brute(f,rranges,pos_var=0,params=[],min_val=100,par_cache=[]):
    if pos_var == len(rranges) - 1:
        for i in rranges[pos_var]:
            params.append(i)
            val = f(params)
            if min_val > val:
                min_val = deepcopy(val)
                par_cache = deepcopy(params)
            params.pop()
        return min_val,par_cache
    else:
        for i in rranges[pos_var]:
            params.append(i)
            min_val,par_cache = rec_brute(f,rranges,pos_var+1,params,min_val,par_cache)
            params.pop()
    return min_val,par_cache

In [4]:
costs = pd.DataFrame({
    'week_1': [4,5,7,1,5],
    'week_2': [6,2,1,3,3],
    'week_3': [3,5,1,6,4]
})
constrains = pd.DataFrame({
    'early_start': [1,1,2,2,3],
    'late_start': [4,3,5,6,5]
})
pd.concat([costs,constrains],axis=1)

,week_1,week_2,week_3,early_start,late_start
0,4,6,3,1,4
1,5,2,5,1,3
2,7,1,1,2,5
3,1,3,6,2,6
4,5,3,4,3,5


In [4]:
n_w_repair = costs.shape[1]
num_weeks = 8
num_mechs = 5
X = np.zeros((num_mechs,num_weeks),dtype='uint8')
Xvar = np.zeros((num_mechs,num_weeks,2),dtype='uint8') # all X variants which we will use for filtering
Xvar[:,:,1] = 1

In [5]:
A = np.arange(1,X.shape[1]+1)

In [6]:
def Ax(Xvar,i,j,mode=np.max):
    return A[:j].dot(mode(Xvar[i,:j],axis=1))+A[j+1:].dot(mode(Xvar[i,j+1:],axis=1))
def Cx(Xvar,i,j,mode=np.max):
    return mode(Xvar[i,:j],axis=1).sum() + mode(Xvar[i,j+1:],axis=1).sum()

In [7]:
def check_W1(Xvar,i,j):
    b1 = A[j]*Xvar[i,j] > constrains['late_start'][i] - Ax(Xvar,i,j,np.min)
    b2 = A[j]*Xvar[i,j] < constrains['early_start'][i] - 1 - Ax(Xvar,i,j,np.max)
    b3 = Xvar[i,j] > 1 - Cx(Xvar,i,j,np.min)
    b4 = Xvar[i,j] < 1 - Cx(Xvar,i,j,np.max)
    return np.r_[[b1,b2,b3,b4]].any(axis=0) # if true by one axis - filter this value

In [8]:
def filter_W1(Xvar):
    Xv = Xvar.copy()
    filtered = False
    for i in range(Xv.shape[0]):
        for j in range(Xv.shape[1]):
            res = check_W1(Xv,i,j)
            if res[0]:
                Xv[i,j] = [1,1]
                filtered = True
            if res[1]:
                Xv[i,j] = [0,0]
                filtered = True
    if filtered:
        return Xv
    else:
        None

$g_p(x)<= g_p^*$   
$min\{g_p(X^p|x_j=x_{j_k})\}>=g^*_p $ - умова відсіювання $x_{j_k}$

In [9]:
# temp
i = 3
j = 3

In [10]:
def W2_check_ij(i,j):
    if Xvar[i,j].mean() == 0.5:
        Xtmp = Xvar.min(axis=2)
        Xtmp[i,j] = 1
        return f_target(Xtmp) > 6
    else:
        return False

In [11]:
class Schedule():
    def __init__(self,week_constraints,costs,num_weeks):
        self.constraints =  week_constraints
        self.costs = costs
        self.n_weeks = num_weeks
        self.n_mechs = costs.shape[0]
        self.n_w_repair = costs.shape[1]
        
    def get_cost(self,X):
        Xar = [X.copy()]
        for i in range(1,self.n_w_repair):
            Xar.append(np.pad(X,((0,0),(i,0)),'constant')[:,:-i])
        return np.c_[
            [ Xar[i].T.dot(self.costs['week_'+str(i+1)]) for i in range(self.n_w_repair) ]
                ].sum(axis=0).max()
    
    def is_valid(self,X):
        if X.shape != (self.n_mechs,self.n_weeks):
            print("invaid shape")
            return False
        if (X.sum(axis=1) != 1).any():
            print("multiple start weeks per machine")
            return False
        
        mask = np.ones_like(X)
        for c in self.constraints.iterrows():
            mask[c[0],c[1]['early_start']-1:c[1]['late_start']-1] = 0
        
        if (X*mask == 0).all():
            return True
        else:
            print("constraints not satisfied")
            return False
    
    def get_cost_w(self,w_starts):
        return np.c_[
                [ np.pad( c, (w_starts[i]-1,
                              self.n_weeks-w_starts[i]+1),
                              'constant') 
                    for i,c in enumerate(costs.values) ]
                ].sum(axis=0).max()
    def apply_early(self,Xvar):
        for i in range(self.constraints['early_start'].max()):
            Xvar[:,:,1][:,i][Xvar[:,:,1][:,i]*(i+1) < self.constraints['early_start']] = 0
        return Xvar
        
    def apply_late(self,Xvar):
        for i in range(self.constraints['late_start'].min()-1,Xvar.shape[1]):
            Xvar[:,:,1][:,i][Xvar[:,:,1][:,i]*(i+1) > self.constraints['late_start']] = 0
        return Xvar
    
    def solve_brute(self):
        rranges = list(self.constraints.apply(lambda x: range(x.early_start,x.late_start+1,1),axis=1))
        return rec_brute(self.get_cost_w,rranges)

In [12]:
repairs = Schedule(constrains,costs,num_weeks=8)

In [13]:
min_val,params = repairs.solve_brute()
print(min_val)

9


In [14]:
def get_costs(params,costs):
    Xt = np.zeros((5,8))
    for c in costs.iterrows():
        for i in range(3):
            Xt[c[0],params[c[0]]-1+i] = c[1]['week_'+str(i+1)]
    print(Xt)
    print(Xt.sum(axis=0).reshape(1,-1))

In [15]:
get_costs(params,costs)

[[ 4.  6.  3.  0.  0.  0.  0.  0.]
 [ 5.  2.  5.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  7.  1.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  3.  6.]
 [ 0.  0.  0.  0.  5.  3.  4.  0.]]
[[ 9.  8.  8.  7.  6.  5.  7.  6.]]


In [16]:
#      _____       _    _           _         _____  _____  _____     
#    |     | ___ | |_ | |_  ___  _| |       |   __||  _  ||   __|    
#   | | | || -_||  _||   || . || . |       |__   ||     ||   __|    
#  |_|_|_||___||_|  |_|_||___||___| _____ |_____||__|__||__|       
#                                  |_____|                         

In [9]:
from ipywidgets import widgets
from IPython.display import display

In [39]:
def number_of_machines(x):
    global n_of_mech
    n_of_mech = x
def number_of_weeks(x):
    global n_of_weeks
    n_of_weeks = x
def get_input_matr(b):
    df = pd.DataFrame(
    np.zeros((n_of_mech,n_of_weeks+2),dtype='uint8'),
    columns=['week_'+str(i) for i in range(1,n_of_weeks+1)] + ['early_start','late_start']
    )
    qgrid_widget = qgrid.QgridWidget(df=df)
    display(qgrid_widget)

def get_test_input(b):
    costs = pd.DataFrame({
        'week_1': [4,5,7,1,5],
        'week_2': [6,2,1,3,3],
        'week_3': [3,5,1,6,4]
    })
    constrains = pd.DataFrame({
        'early_start': [1,1,2,2,3],
        'late_start': [4,3,5,6,5]
    })
    df = pd.concat([costs,constrains],axis=1)
    qgrid_widget = qgrid.QgridWidget(df=df)
    display(qgrid_widget)

widgets.interact(number_of_machines, x=widgets.IntSlider(min=0,max=10,step=1,value=5))
widgets.interact(number_of_weeks, x=widgets.IntSlider(min=0,max=20,step=1,value=8))
b_matr= widgets.Button(description='GET EM')
b_matr.on_click(get_input_matr)
display(b_matr)
b_test = widgets.Button(description='Test Case')
b_test.on_click(get_test_input)
display(b_test)

interactive(children=(IntSlider(value=5, description='x', max=10), Output()), _dom_classes=('widget-interact',))

interactive(children=(IntSlider(value=8, description='x', max=20), Output()), _dom_classes=('widget-interact',))

Button(description='GET EM', style=ButtonStyle())

Button(description='Test Case', style=ButtonStyle())

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defaultColumnWidth': 150, 'rowHeight': 28, 'enableColumnReorder': False, 'enableTextSelectionOnCells': True, 'editable': True, 'autoEdit': False, 'explicitInitialization': True, 'maxVisibleRows': 15, 'minVisibleRows': 8}, precision=5)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defaultColumnWidth': 150, 'rowHeight': 28, 'enableColumnReorder': False, 'enableTextSelectionOnCells': True, 'editable': True, 'autoEdit': False, 'explicitInitialization': True, 'maxVisibleRows': 15, 'minVisibleRows': 8}, precision=5)

In [46]:
qgrid_widget.

TypeError: notify_change() missing 1 required positional argument: 'change'